# Notebook 4: Efficient Location and Time Series Access Using Polytope Feature Extraction

Notebook 4 showcases efficient geo-location and time series access using polytope feature extraction. By leveraging this feature, we significantly reduce data retrieval from storage, optimizing workflows and resource utilization.

1. **Configuring Access to Polytope**

In [ ]:
import os
import dataclasses as dc
from meteodatalab import mars

os.environ["POLYTOPE_USER_KEY"] = "xxxxxxxxxxxxxxxxx"
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-dev.mchml.cscs.ch"


2. **Selecting geo-location point of interest**

In [ ]:
zrh = mars.Point(47.47, 8.55)

3. **Querying the data** <br>
The `feature` attribute tells polytope to extract **only the relevant data at the given point**.
The amount of data that is retrieved from storage is signficantly reduced.

In [ ]:
request = mars.Request(
    param="T_2M",
    date="20240222",
    time="0300",
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    number=1,
    feature=mars.TimeseriesFeature(
        points=[zrh],
        range=mars.Range(start=0, end=30),
        time_axis="step",
    )
)

In [ ]:
from meteodatalab import mch_model_data

ds = mch_model_data.get_from_polytope(request)

4. **Visualizing the data**

In [ ]:
ds["t_2m"].plot()

5. **Concatenating timeseries**

In [ ]:
# mars.Request is a dataclass
requests = [dc.replace(request, date=str(d), time="0000") for d in range(20240414, 20240425)]
requests

In [ ]:
from concurrent.futures import ThreadPoolExecutor

import xarray as xr

with ThreadPoolExecutor(max_workers=5) as exec:
    ds = list(exec.map(mch_model_data.get_from_polytope, requests))

t_2m = xr.concat([a["2m Temperature"] for a in ds], dim="t")

t_2m.plot()


# Source: https://nwp-polytope-test.s3.eu-central-2.amazonaws.com/5ddc3f38-2b49-4e49-9f4f-c2287d85e15c
